In [4]:
#uploading data
dataset = read.csv("wageit.csv")

In [6]:
View(dataset)

ERROR: Error in View(dataset): 'View()' not yet supported in the Jupyter R kernel


In [7]:
print(dataset)

     female cg sc hsg mw so we ne exp1    exp2      exp3        wage
1         0  0  0   1  0  0  0  1 33.0 10.8900 35.937000  11.6590909
2         0  1  0   0  0  0  0  1 27.0  7.2900 19.683000  12.8250000
3         0  0  1   0  0  0  0  1 13.0  1.6900  2.197000   5.7770270
4         0  1  0   0  0  0  0  1  2.0  0.0400  0.008000  12.4687500
5         1  1  0   0  0  0  0  1 15.0  2.2500  3.375000  18.5250000
6         0  0  1   0  0  0  0  1  6.5  0.4225  0.274625  11.4000000
7         1  0  1   0  0  0  0  1  6.0  0.3600  0.216000   7.7751987
8         0  0  0   1  0  0  0  1 25.0  6.2500 15.625000  16.0312500
9         0  0  0   1  0  0  0  1 14.0  1.9600  2.744000   9.7500000
10        0  0  1   0  0  0  0  1 26.0  6.7600 17.576000   5.4557143
11        0  0  0   1  0  0  0  1 25.0  6.2500 15.625000  12.4687500
12        1  1  0   0  0  0  0  1  4.0  0.1600  0.064000  15.7500000
13        0  0  0   1  0  0  0  1  7.0  0.4900  0.343000   7.1250000
14        1  0  1   0  0  0  0  1 

In [10]:
# getting mean for each variable(2 here is to take column mean)
variable_average = as.matrix(apply(dataset, 2, mean))
colnames(variable_average) = c("Mean/Average")

In [12]:
print(variable_average)

       Mean/Average
female    0.4179922
cg        0.3762712
sc        0.3238592
hsg       0.2998696
mw        0.2876141
so        0.2435463
we        0.2117340
ne        0.2571056
exp1     13.3531943
exp2      2.5292666
exp3      5.8121030
wage     15.5333559


In [13]:
# Compute Linear Regression Model 
regressor1 =  wage ~ female + sc+ cg+ mw + so + we + exp1 + exp2 + exp3

In [14]:
# Compute Linear Model and  MSE and R-Squared
full.lm1 = lm(regressor1, data=dataset)
lm1 = summary(full.lm1)
R2.1 = lm1$r.squared
R2.adj1 = lm1$adj.r.squared
n1 =  length(lm1$res)
p1  = lm1$df[1]
MSE.adj1  = (n1/(n1-p1))*mean(lm1$res^2)

In [15]:
# Compute Linear Regression Model (with quadratic specification)
regressor2 = wage ~  female + (sc+ cg+ mw + so + we + exp1 + exp2 + exp3)^2

In [16]:
# Compute Linear Model and  MSE and R-Squared (with quadratic specification)
full.lmQ = lm(regressor2, data=dataset)
lmQ = summary(full.lmQ)
R2.2  = lmQ$r.squared
R2.adj2 = lmQ$adj.r.squared
n2  = length(lmQ$res)
p2  = lmQ$df[1]
MSE.adj2  = (n2/(n2-p2))*mean(lmQ$res^2)

In [17]:
# Summary Stat for linear and quadratic specifications
table1     = matrix(0, 2, 4)
table1[1,] = c(p1, R2.1, R2.adj1, MSE.adj1)
table1[2,] = c(p2, R2.2, R2.adj2, MSE.adj2)  

# Show Regresssion Results
colnames(table1) = c("p", "R^2", "R^2 adj", "MSE adj")
rownames(table1) = c("basic reg", "flex reg")

In [18]:
print(table1)

           p       R^2    R^2 adj  MSE adj
basic reg 10 0.0954880 0.09335974 165.6802
flex reg  33 0.1039728 0.09643130 165.1189


# Linear and Quadratic specifications with Sample Splitting

In [19]:
# set random number generator
set.seed(123)

In [20]:
# Split dataset into training and test sample(50/50 Ratio)
train = sample(1:nrow(dataset), nrow(dataset)/2)

In [21]:
# Compute linear specification and compute MSE and R^2 for the test sample
full.lm1  = lm(regressor1, data=dataset[train,])
yhat.lm1  = predict(full.lm1, newdata=dataset[-train,])
y.test = dataset[-train,]$wage
MSE.lm1  = summary(lm((y.test-yhat.lm1)^2~1))$coef[1]
R2.lm1 = 1- MSE.lm1/var(y.test)

In [22]:
# Split dataset into training and test sample
train = sample(1:nrow(dataset), nrow(dataset)/2)

In [24]:
# run quadratic specification and compute MSE and R^2 for the test sample
full.lmQ  = lm(regressor2, data=dataset[train,])
yhat.lmQ  = predict(full.lmQ, newdata=dataset[-train,])	
y.test = dataset[-train,]$wage
MSE.lmQ = summary(lm((y.test-yhat.lmQ)^2~1))$coef[1]
R2.lmQ  = 1- MSE.lmQ/var(y.test)

Warning message in predict.lm(full.lmQ, newdata = dataset[-train, ]):
"prediction from a rank-deficient fit may be misleading"

In [26]:
# Create result table
table2      = matrix(0, 2, 3)
table2[1,]  = c(p1, R2.lm1, MSE.lm1)
table2[2,]  = c(p2, R2.lmQ, MSE.lmQ)

# Give Columns and Row Names
colnames(table2)  = c("p ", "R2 test", "MSE test")
rownames(table2)  = c("basic reg", "flex reg")

In [28]:
print(table2)

          p    R2 test MSE test
basic reg 10 0.0840296 129.2131
flex reg  33 0.1182428 118.7135


In [29]:
# Show Results
print(table1,digits=4)
print(table2,digits=4)

           p     R^2 R^2 adj MSE adj
basic reg 10 0.09549 0.09336   165.7
flex reg  33 0.10397 0.09643   165.1
          p  R2 test MSE test
basic reg 10 0.08403    129.2
flex reg  33 0.11824    118.7
